In [ ]:
# download data from kaggle
!curl -L -o image-classification.zip https://www.kaggle.com/api/v1/datasets/download/duttadebadri/image-classification
# unzip downloaded data
!unzip image-classification.zip -d images

In [ ]:
import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

image_files = glob.glob("images/*/*jpg")
architecure_files = glob.glob("images/architecure/*jpg")
art_culture_files = glob.glob("images/art and culture/*jpg")
food_drinks_files = glob.glob("images/food and d rinks/*jpg")
travel_adventure_files = glob.glob("images/travel and  adventure/*jpg")

print("number of architecure images:", len(architecure_files))  
print("number of art_culture images:", len(art_culture_files))
print("number of food_drinks_ images:", len(food_drinks_files))
print("number of travel_adventure images:", len(travel_adventure_files))
print("number of images:", len(image_files))

In [20]:
test_ratio = 0.2
train_arch, test_arch = architecure_files[:int(len(architecure_files)*(1-test_ratio))], architecure_files[-int(len(architecure_files)*(test_ratio)):]
train_art, test_art = art_culture_files[:int(len(art_culture_files)*(1-test_ratio))], art_culture_files[-int(len(art_culture_files)*(test_ratio)):]
train_food, test_food = food_drinks_files[:int(len(food_drinks_files)*(1-test_ratio))], food_drinks_files[-int(len(food_drinks_files)*(test_ratio)):]
train_travel, test_travel = travel_adventure_files[:int(len(travel_adventure_files)*(1-test_ratio))], travel_adventure_files[-int(len(travel_adventure_files)*(test_ratio)):]


In [35]:
for train_files in [train_arch, train_art, train_food, train_travel]:
    for file in train_files:
        # Get folder name from file path
        folder_name = os.path.basename(os.path.dirname(file))
        path = os.path.join('images', 'train', folder_name, file.split("\\")[-1])
        # Create train directory and subdirectory if not exists
        os.makedirs(path, exist_ok=True)
        
        # # Copy file to train directory
        os.system(f'copy "{file}" "{path}"')

for test_files in [test_arch, test_art, test_food, test_travel]:
    for file in test_files:
        # Get folder name from file path
        folder_name = os.path.basename(os.path.dirname(file))
        path = os.path.join('images', 'test', folder_name, file.split("\\")[-1])
        # Create train directory and subdirectory if not exists
        os.makedirs(path, exist_ok=True)
        
        # # Copy file to train directory
        os.system(f'copy "{file}" "{path}"')

In [ ]:


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')  # 2 kelas output, sesuaikan dengan kebutuhan
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    'images/train', 
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'images/test',  
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)


In [ ]:
# Membuat plot history training
import matplotlib.pyplot as plt

# Plot akurasi training dan validasi
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss training dan validasi
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
train_datagen

In [ ]:
model.save('image_classification_model.h5')

In [ ]:
architecure_files = glob.glob("images/architecure/*jpg")
art_culture_files = glob.glob("images/art and culture/*jpg")
food_drinks_files = glob.glob("images/food and d rinks/*jpg")
travel_adventure_files = glob.glob("images/travel and  adventure/*jpg")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load model
model = load_model('image_classification_model.h5')
class_names = ["architecure", "art_culture", "food_drinks", "travel_adventure"]
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    
    return predicted_class, predictions[0]

img_path = 'example.jpg' 
predicted_class, probabilities = predict_image(img_path)

print(f'Predicted class: {predicted_class}')
print(f'Class probability: {probabilities}')

